# Instructions
1. Load the houseprices data from Thinkful's database.
2. Do data cleaning, exploratory data analysis, and feature engineering. You can use your previous work in this module. But make sure that your work is satisfactory.
3. Now, split your data into train and test sets where 20% of the data resides in the test set.
4. Build several linear regression models including Lasso, Ridge, or ElasticNet and train them in the training set. Use k-fold cross-validation to select the best hyperparameters if your models include one!
5. Evaluate your best model on the test set.
6. So far, you have only used the features in the dataset. However, house prices can be affected by many factors like economic activity and the interest rates at the time they are sold. So, try to find some useful factors that are not included in the dataset. Integrate these factors into your model and assess the prediction performance of your model. Discuss the implications of adding these external variables into your model.


### Start Notebook environment

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy import create_engine

In [2]:
# %load ../utility/overhead.py
#record module versions used in cell 1
#
def version_recorder():
    '''
    only works if import is first cell run. prints and then returns dictionary with modules:version.
    '''
    import pkg_resources
    resources = In[1].splitlines()
    ##ADD: drop lines if not _from_ or _import_
    version_dict = { resource.split()[1].split(".")[0] : pkg_resources.get_distribution(resource.split()[1].split(".")[0]).version for resource in resources }
    return version_dict
version_recorder()

{'numpy': '1.16.2',
 'pandas': '0.24.2',
 'sklearn': '0.0',
 'matplotlib': '3.0.3',
 'seaborn': '0.9.0',
 'sqlalchemy': '1.3.5'}

# Data Cleaning

In [3]:
#credentials
user = 'dsbc_student'
pw = '7*.8G9QH21'
host = '142.93.121.174'
port = '5432'
db = 'houseprices'
dialect = 'postgresql'

db_addr = f'{dialect}://{user}:{pw}@{host}:{port}/{db}'
engine = create_engine(db_addr)

query = '''
SELECT
    *
FROM
    houseprices
'''

raw_df = pd.read_sql(query, con=engine)
engine.dispose()

In [4]:
house_df = raw_df.copy(deep=True)
house_df.head()

,id,mssubclass,mszoning,lotfrontage,lotarea,street,alley,lotshape,landcontour,utilities,...,poolarea,poolqc,fence,miscfeature,miscval,mosold,yrsold,saletype,salecondition,saleprice
0,1,60,RL,65.0,8450,Pave,None,Reg,Lvl,AllPub,...,0,None,None,None,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,None,Reg,Lvl,AllPub,...,0,None,None,None,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,None,IR1,Lvl,AllPub,...,0,None,None,None,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,None,IR1,Lvl,AllPub,...,0,None,None,None,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,None,IR1,Lvl,AllPub,...,0,None,None,None,0,12,2008,WD,Normal,250000


In [5]:
house_df.columns

Index(['id', 'mssubclass', 'mszoning', 'lotfrontage', 'lotarea', 'street',
       'alley', 'lotshape', 'landcontour', 'utilities', 'lotconfig',
       'landslope', 'neighborhood', 'condition1', 'condition2', 'bldgtype',
       'housestyle', 'overallqual', 'overallcond', 'yearbuilt', 'yearremodadd',
       'roofstyle', 'roofmatl', 'exterior1st', 'exterior2nd', 'masvnrtype',
       'masvnrarea', 'exterqual', 'extercond', 'foundation', 'bsmtqual',
       'bsmtcond', 'bsmtexposure', 'bsmtfintype1', 'bsmtfinsf1',
       'bsmtfintype2', 'bsmtfinsf2', 'bsmtunfsf', 'totalbsmtsf', 'heating',
       'heatingqc', 'centralair', 'electrical', 'firstflrsf', 'secondflrsf',
       'lowqualfinsf', 'grlivarea', 'bsmtfullbath', 'bsmthalfbath', 'fullbath',
       'halfbath', 'bedroomabvgr', 'kitchenabvgr', 'kitchenqual',
       'totrmsabvgrd', 'functional', 'fireplaces', 'fireplacequ', 'garagetype',
       'garageyrblt', 'garagefinish', 'garagecars', 'garagearea', 'garagequal',
       'garagecond', 'paved

In [6]:
house_df.columns[house_df.dtypes != 'object']

Index(['id', 'mssubclass', 'lotfrontage', 'lotarea', 'overallqual',
       'overallcond', 'yearbuilt', 'yearremodadd', 'masvnrarea', 'bsmtfinsf1',
       'bsmtfinsf2', 'bsmtunfsf', 'totalbsmtsf', 'firstflrsf', 'secondflrsf',
       'lowqualfinsf', 'grlivarea', 'bsmtfullbath', 'bsmthalfbath', 'fullbath',
       'halfbath', 'bedroomabvgr', 'kitchenabvgr', 'totrmsabvgrd',
       'fireplaces', 'garageyrblt', 'garagecars', 'garagearea', 'wooddecksf',
       'openporchsf', 'enclosedporch', 'threessnporch', 'screenporch',
       'poolarea', 'miscval', 'mosold', 'yrsold', 'saleprice'],
      dtype='object')

In [7]:
house_df.corr()["saleprice"].abs().sort_values(ascending=False)

saleprice        1.000000
overallqual      0.790982
grlivarea        0.708624
garagecars       0.640409
garagearea       0.623431
totalbsmtsf      0.613581
firstflrsf       0.605852
fullbath         0.560664
totrmsabvgrd     0.533723
yearbuilt        0.522897
yearremodadd     0.507101
garageyrblt      0.486362
masvnrarea       0.477493
fireplaces       0.466929
bsmtfinsf1       0.386420
lotfrontage      0.351799
wooddecksf       0.324413
secondflrsf      0.319334
openporchsf      0.315856
halfbath         0.284108
lotarea          0.263843
bsmtfullbath     0.227122
bsmtunfsf        0.214479
bedroomabvgr     0.168213
kitchenabvgr     0.135907
enclosedporch    0.128578
screenporch      0.111447
poolarea         0.092404
mssubclass       0.084284
overallcond      0.077856
mosold           0.046432
threessnporch    0.044584
yrsold           0.028923
lowqualfinsf     0.025606
id               0.021917
miscval          0.021190
bsmthalfbath     0.016844
bsmtfinsf2       0.011378
Name: salepr

In [8]:
#create df with categorical variables to select some features from
categorical_feat = house_df.dtypes[house_df.dtypes == 'object'].index
new_categories_df = house_df[["saleprice"]]
for feature in categorical_feat:
    new_categories_df = pd.concat([new_categories_df, 
                                   pd.get_dummies(house_df[feature], columns=categorical_feat, drop_first=True, prefix = feature)], axis=1)

In [9]:
new_categories_df.corr()[["saleprice"]].abs().sort_values(by='saleprice')

,saleprice
bsmtfintype2_GLQ,0.000076
roofmatl_Metal,0.000304
roofstyle_Mansard,0.000308
garagecond_Gd,0.000983
foundation_Wood,0.002711
condition2_RRAe,0.002993
bldgtype_TwnhsE,0.003804
condition1_RRNe,0.004584
roofmatl_Tar&Grv,0.004921
condition1_RRAn,0.005893


In [10]:
#append numerical features to new df
new_categories_df = pd.concat([new_categories_df, 
                               house_df.filter(items=(house_df.columns[(house_df.dtypes.values != 'object').tolist()]), axis=1) ], 
                              axis=1) #tolist() needed to avoid hashability issue

#create X & y
X = house_df[["overallqual", "grlivarea", "fullbath", "yearbuilt", "yearremodadd", "garagecars", "fireplaces", "yrsold"]]
X = pd.concat([X, new_categories_df[['exterqual_TA', 'foundation_CBlock']]], 1)
X["overalqual_x_year"] = house_df.overallqual * house_df.yearbuilt
y = house_df.saleprice

# Model Comparison
## Split data

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.15)

## Lasso

In [12]:
lasso_model = Lasso(tol=.5)  #increased tol to avoid 'ConvergenceWarning: Objective did not converge'
alphas = np.array([.0001, .001, .01, .1, 1, 10, 100, 1_000, 10_000, 100_000])

grid_las = GridSearchCV(estimator=lasso_model, param_grid=dict(alpha=alphas), cv=12)
grid_las.fit(X_train, y_train)
print(grid_las)
print("best r^2 is: {}".format(grid_las.best_score_))
print("associated lambda value is: {}".format(grid_las.best_estimator_.alpha))

GridSearchCV(cv=12, error_score='raise-deprecating',
             estimator=Lasso(alpha=1.0, copy_X=True, fit_intercept=True,
                             max_iter=1000, normalize=False, positive=False,
                             precompute=False, random_state=None,
                             selection='cyclic', tol=0.5, warm_start=False),
             iid='warn', n_jobs=None,
             param_grid={'alpha': array([1.e-04, 1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03,
       1.e+04, 1.e+05])},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)
best r^2 is: 0.715835138772405
associated lambda value is: 1000.0


c:\users\chris\appdata\local\programs\python\python37\lib\site-packages\sklearn\model_selection\_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


## Ridge

In [13]:
ridge_model = Ridge()
alphas = np.array([.0001, .001, .01, .1, 1, 10, 100, 1_000, 10_000, 100_000])

grid_ridge = GridSearchCV(estimator=ridge_model, param_grid=dict(alpha=alphas), cv=7)
grid_ridge.fit(X_train, y_train)
print(grid_ridge)
print("best r^2 is: {}".format(grid_ridge.best_score_))
print("associated lambda value is: {}".format(grid_ridge.best_estimator_.alpha))

GridSearchCV(cv=7, error_score='raise-deprecating',
             estimator=Ridge(alpha=1.0, copy_X=True, fit_intercept=True,
                             max_iter=None, normalize=False, random_state=None,
                             solver='auto', tol=0.001),
             iid='warn', n_jobs=None,
             param_grid={'alpha': array([1.e-04, 1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03,
       1.e+04, 1.e+05])},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)
best r^2 is: 0.7320212860381149
associated lambda value is: 0.1


## ElasticNet

In [14]:
elasticnet_model = ElasticNet(tol=1)   #increased tol to avoid 'ConvergenceWarning: Objective did not converge'
alphas = np.array([.0001, .001, .01, .1, 1, 10, 100])
l1_ratio = np.array([.1, .5, .9])
params = {'alpha': alphas, 'l1_ratio': l1_ratio}

grid_enet = GridSearchCV(estimator=elasticnet_model, param_grid=params, cv=7)
grid_enet.fit(X_train, y_train)
print(grid_enet)
print("best r^2 is: {}".format(grid_enet.best_score_))
print("associated lambda value is: {}".format(grid_enet.best_estimator_.alpha))
print("associated estimator value is: {}".format(grid_enet.best_estimator_))

GridSearchCV(cv=7, error_score='raise-deprecating',
             estimator=ElasticNet(alpha=1.0, copy_X=True, fit_intercept=True,
                                  l1_ratio=0.5, max_iter=1000, normalize=False,
                                  positive=False, precompute=False,
                                  random_state=None, selection='cyclic', tol=1,
                                  warm_start=False),
             iid='warn', n_jobs=None,
             param_grid={'alpha': array([1.e-04, 1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02]),
                         'l1_ratio': array([0.1, 0.5, 0.9])},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)
best r^2 is: 0.7065672728751776
associated lambda value is: 1.0
associated estimator value is: ElasticNet(alpha=1.0, copy_X=True, fit_intercept=True, l1_ratio=0.9,
           max_iter=1000, normalize=False, positive=False, precompute=False,
           random_state=None, selec

# External Factors

Realtors may cite Location, Location, Location, but timing is vital when buying and selling assets. The Great Recession had a huge impact on the housing market (and vice-versa.) I am including a feature for the economic condition: annual unemployment rate (monthly is available, but would introduce difficulties in comparing seasonality for house sales vs employment)

I have also included interest rate. Most houses are purchased via mortgage and so, for a given monthly budget, a higher interest rate will result in a lower possible purchase price budget.

Citations:
Organization for Economic Co-operation and Development, Unemployment Rate: Aged 15-64: All Persons for the United States [LRUN64TTUSA156S], retrieved from FRED, Federal Reserve Bank of St. Louis; https://fred.stlouisfed.org/series/LRUN64TTUSA156S, August 6, 2019.

Freddie Mac, 30-Year Fixed Rate Mortgage Average in the United States [MORTGAGE30US], retrieved from FRED, Federal Reserve Bank of St. Louis; https://fred.stlouisfed.org/series/MORTGAGE30US, August 7, 2019.

In [15]:
econ_df = pd.read_csv(r'../../../data_sets/LRUN64TTUSA156S.csv')

In [16]:
econ_df.DATE = pd.to_datetime(econ_df["DATE"])

In [17]:
econ_df["year"] = econ_df.DATE.dt.year
econ_df.head()

,DATE,LRUN64TTUSA156S,year
0,1960-01-01,5.617115,1960
1,1961-01-01,6.770271,1961
2,1962-01-01,5.596210,1962
3,1963-01-01,5.739126,1963
4,1964-01-01,5.246175,1964


In [18]:
econ2_df = pd.read_csv(r'../../../data_sets/MORTGAGE30US.csv')

In [19]:
econ2_df.head()

,DATE,MORTGAGE30US
0,1971-04-02,7.33
1,1971-04-09,7.31
2,1971-04-16,7.31
3,1971-04-23,7.31
4,1971-04-30,7.29


In [20]:
#get annual interest rate average
econ2_df["year"] = pd.to_datetime(econ2_df.DATE).dt.year
econ_rates_df = econ2_df.groupby("year").mean()
econ_rates_df.head()

,MORTGAGE30US
year,
1971,7.541750
1972,7.383269
1973,8.044808
1974,9.187115
1975,9.047115


In [21]:
econ_both_df = pd.merge(left=econ_df, right=econ_rates_df, on='year').drop(["DATE"], axis=1)
econ_both_df = econ_both_df.rename(columns={'LRUN64TTUSA156S':'unemployment', 'MORTGAGE30US':'mtg_rate'})
econ_both_df.head()

,unemployment,year,mtg_rate
0,6.041505,1971,7.541750
1,5.688501,1972,7.383269
2,4.950751,1973,8.044808
3,5.677059,1974,9.187115
4,8.561464,1975,9.047115


In [22]:
#combine econ data with house data 
X_wecon_train = pd.merge(left=X_train, right=econ_both_df, left_on='yrsold', right_on='year', how='left')
X_wecon_test = pd.merge(left=X_test, right=econ_both_df, left_on='yrsold', right_on='year', how='left')

In [24]:
elasticnet_model2 = ElasticNet(tol=1)   #increased tol to avoid 'ConvergenceWarning: Objective did not converge'
alphas = np.array([.0001, .001, .01, .1, 1, 10, 100])
l1_ratio = np.array([.1, .5, .9])
params = {'alpha': alphas, 'l1_ratio': l1_ratio}

grid_enet2 = GridSearchCV(estimator=elasticnet_model2, param_grid=params, cv=7)
grid_enet.fit(X_wecon_train, y_train)
print(grid_enet)
print("best r^2 is: {}".format(grid_enet.best_score_))
print("associated lambda value is: {}".format(grid_enet.best_estimator_.alpha))
print("associated estimator value is: {}".format(grid_enet.best_estimator_))

GridSearchCV(cv=7, error_score='raise-deprecating',
             estimator=ElasticNet(alpha=1.0, copy_X=True, fit_intercept=True,
                                  l1_ratio=0.5, max_iter=1000, normalize=False,
                                  positive=False, precompute=False,
                                  random_state=None, selection='cyclic', tol=1,
                                  warm_start=False),
             iid='warn', n_jobs=None,
             param_grid={'alpha': array([1.e-04, 1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02]),
                         'l1_ratio': array([0.1, 0.5, 0.9])},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)
best r^2 is: 0.7063433873790307
associated lambda value is: 1.0
associated estimator value is: ElasticNet(alpha=1.0, copy_X=True, fit_intercept=True, l1_ratio=0.9,
           max_iter=1000, normalize=False, positive=False, precompute=False,
           random_state=None, selec